# KolmoLabs Business Agent Notebook

This notebook mirrors the Gradio app in `app.py`, letting you iterate on the UI or run quick regression checks without leaving Jupyter.


## 1. Imports & environment setup

We reload `agent_core` to pick up the latest tool definitions before importing helper functions and paths.


In [ ]:
from pathlib import Path
from typing import List, Tuple

import importlib
import gradio as gr

import agent_core
agent_core = importlib.reload(agent_core)

from agent_core import chat_once, new_conversation, LEADS_CSV, DEMO_REQUESTS_CSV, PHONE_LEADS_CSV, FEEDBACK_LOG

LOGO_PATH = Path("Logo.jpg")


## 2. Helper functions & UI factory

The code below is copy-pasted from `app.py` so any UI changes there stay in sync here.


In [ ]:
def _agent_turn(agent_history: List[dict], user_message: str) -> Tuple[List[dict], str]:
    """Run a single agent turn and return updated history + reply."""
    return chat_once(agent_history, user_message)

def build_demo() -> gr.Blocks:
    """Return a configured Gradio Blocks instance for the chatbot."""
    theme = gr.themes.Soft(
        primary_hue="violet",
        neutral_hue="slate",
        radius_size="lg",
        spacing_size="sm",
    )
    css = """
    body {
        background: linear-gradient(180deg, #f5f3ff 0%, #fdfcff 100%);
    }
    .gradio-container {
        max-width: 900px !important;
        margin: 0 auto;
        padding: 0.5rem 1rem !important;
    }
    .contain, .gr-box, .gr-form, .gr-group {
        padding: 0 !important;
        margin: 0 !important;
        gap: 0.25rem !important;
    }
    .gr-box h1 {
        margin: 0.5rem 0 0.25rem 0 !important;
        padding: 0 !important;
        font-size: 1.5rem !important;
        line-height: 1.2 !important;
    }
    .gr-box p {
        margin: 0 0 0.5rem 0 !important;
        padding: 0 !important;
        font-size: 0.9rem !important;
        line-height: 1.3 !important;
    }
    .gr-chatbot {
        border-radius: 14px !important;
        border: 1px solid rgba(96, 78, 255, 0.15);
        box-shadow: 0 4px 16px rgba(42, 34, 94, 0.08);
        background: white;
        margin: 0.5rem 0 !important;
    }
    .gr-chatbot .message {
        border-radius: 12px !important;
        padding: 0.5rem 0.75rem !important;
    }
    .gr-textbox {
        margin: 0.5rem 0 !important;
    }
    .gr-textbox textarea {
        min-height: 60px !important;
        font-size: 0.95rem;
        border-radius: 12px !important;
        padding: 0.5rem !important;
    }
    .gr-button {
        border-radius: 10px !important;
        padding: 0.4rem 1rem !important;
        margin: 0.25rem 0 !important;
    }
    .gradio-container footer {
        margin-top: 0.5rem !important;
        padding: 0.5rem !important;
        box-shadow: none !important;
    }
    .gr-form > * {
        margin-bottom: 0 !important;
    }
    """

    with gr.Blocks(title="KolmoLabs Business Assistant", theme=theme, css=css) as demo:
        agent_state = gr.State(new_conversation())

        def respond(message: str, history, agent_history):
            if not message.strip():
                return "", agent_history
            new_agent_history, reply = _agent_turn(agent_history, message)
            return reply, new_agent_history

        def respond_stream(message: str, history, agent_history):
            reply, updated_history = respond(message, history, agent_history)
            yield reply, updated_history

        chatbot_component = gr.Chatbot(
            label="KolmoLabs Assistant",
            type="messages",
            height=550,
            avatar_images=(
                None,
                str(LOGO_PATH.resolve()) if LOGO_PATH.exists() else None,
            ),
            show_copy_button=True,
        )

        chat = gr.ChatInterface(
            fn=respond_stream,
            type="messages",
            chatbot=chatbot_component,
            textbox=gr.Textbox(
                placeholder="Ask about KolmoLabs' mission, services, pricing, or partnerships...",
                lines=1,
                max_lines=3,
                autofocus=True,
                submit_btn="Send",
                stop_btn="Stop",
            ),
            additional_inputs=[agent_state],
            additional_outputs=[agent_state],
            title="KolmoLabs Business Assistant",
            description="Fast answers about Kolmogorov Neural Network services for MENA SMBs. Happy to capture leads, schedule demos, or note phone call requests.",
            theme=theme,
        )

        chat.chatbot.clear(

            lambda: new_conversation(),
            outputs=[agent_state],
            queue=False,
            show_api=False,
        )

    return demo


## 3. Launch the Gradio app

Run the next cell to spin up the demo exactly like `python app.py`. Interrupt the cell or press the app's “Stop” button to shut it down.


In [ ]:
demo = build_demo()
demo.launch(server_name="127.0.0.1", server_port=5000, share=False, show_api=False)


## 4. Optional: CLI-style regression probes

This cell exercises the assistant without the UI and previews the tail of each log file. Uncomment lines to trigger specific tool paths.


In [ ]:
from pathlib import Path

def preview_file(label: str, path: Path, tail: int = 5):
    print(f"{label}: {path}")
    if not path.exists():
        print("  - no entries yet\n")
        return
    lines = [line for line in path.read_text(encoding="utf-8").strip().splitlines() if line]
    if not lines:
        print("  - empty file\n")
        return
    print("  - showing last entries:")
    for line in lines[-tail:]:
        print(f"    {line}")
    print()

hist = new_conversation()
for prompt in ["What is your mission?", "List two flagship Kolmogorov Neural Network services you provide."]:
    hist, response = chat_once(hist, prompt)
    print("Q:", prompt)
    print("A:", response)
    print("-" * 60)

# Uncomment to trigger tool calls quickly:
# chat_once(new_conversation(), "I'm Leen at HorizonX. leen@hx.ai — let's talk AI ops.")
# chat_once(new_conversation(), "Need a KolmoLabs demo next Wednesday 14:00 Beirut. I'm Maya, maya@cedartech.io.")
# chat_once(new_conversation(), "Call me tonight at +971-50-1112233 about anomaly detection. I'm Tareq.")
# chat_once(new_conversation(), "Can you share your Dubai office address and enterprise pricing tiers?")

preview_file("Leads", Path(LEADS_CSV))
preview_file("Demo requests", Path(DEMO_REQUESTS_CSV))
preview_file("Phone contacts", Path(PHONE_LEADS_CSV))
preview_file("Feedback log", Path(FEEDBACK_LOG))


## 5. Quick testing checklist

- **Knowledge answers (no tool):** `What is your mission?`
- **Lead capture → `leads.csv`:** `I'm Karim from Orbit Labs. Email karim@orbit.ai—could someone reach out?`
- **Demo request → `demo_requests.csv`:** `I need a KolmoLabs demo next Tuesday 15:00 Beirut. I'm Salma Haddad, salma.haddad@cedarcapital.ai.`
- **Phone follow-up → `phone_contacts.csv`:** `Call me tomorrow at +961-3-555555. I'm Omar Kassab and need onboarding help.`
- **Feedback escalation → `feedback.log`:** `Can you share your Dubai office address and enterprise pricing tiers?`
